In [21]:
#To install requirements use pip-chill -r requirements.txt

from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [19]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
# 1536 are dimensions for OpenAI embedding model. 
# If we switch, we need to make sure the dimensions match the open source embedding model
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

5a31e555-14b2-44bc-b0dd-1f01ef3217a8


In [20]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'review': "Dr. Johnson's lectures are engaging and insightful. She makes complex topics easy to understand.",
  'subject': 'Physics',
  'stars': 5},
 {'professor': 'Prof. Mark Thompson',
  'review': 'Prof. Thompson is knowledgeable but often goes off-topic. His grading can be quite strict.',
  'subject': 'Mathematics',
  'stars': 3},
 {'professor': 'Dr. Sarah Lee',
  'review': 'Dr. Lee is very approachable and always willing to help outside of class. Her assignments are challenging but fair.',
  'subject': 'Computer Science',
  'stars': 4},
 {'professor': 'Dr. John Smith',
  'review': "Dr. Smith's lectures are dry and monotonous. The course materials are outdated.",
  'subject': 'History',
  'stars': 2},
 {'professor': 'Prof. Linda Carter',
  'review': 'Prof. Carter has a great teaching style and is very enthusiastic about the subject. The exams are quite difficult.',
  'subject': 'Literature',
  'stars': 4},
 {'professor': 'Dr. Alex Brown',
  'rev

In [22]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    #This code would need to be changed for other models
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }

    })

In [23]:
processed_data[0]

{'values': [0.01570056,
  -0.0048187687,
  0.017587109,
  0.021397443,
  0.016221844,
  0.00087346,
  -0.0016522822,
  0.06657534,
  0.008948699,
  0.0054207267,
  0.041975725,
  0.0013257044,
  -0.02688333,
  0.022762708,
  0.0099354135,
  0.01841868,
  -0.052326925,
  -0.023743218,
  0.06692286,
  0.071043484,
  0.056795068,
  -0.018182863,
  0.03033373,
  -0.012510802,
  -0.020292819,
  -0.04855383,
  0.015266157,
  0.009817504,
  0.031128068,
  -0.008731497,
  0.057738345,
  0.0043998803,
  -0.0404367,
  -0.024872666,
  -0.005591385,
  0.02351981,
  -0.0028174128,
  -0.009718212,
  0.013131377,
  0.010208467,
  0.02384251,
  -0.010369817,
  -0.03723453,
  -3.4955854e-05,
  0.057738345,
  0.021000275,
  0.013205847,
  -0.018865494,
  0.04890135,
  0.052326925,
  -0.038351566,
  0.010655281,
  0.009848533,
  -0.014198767,
  -0.036117494,
  -0.021260917,
  -0.0018617265,
  0.04867794,
  -0.04068493,
  -0.022713063,
  0.032245103,
  -0.0020758251,
  -0.014757285,
  -2.4810899e-05,
  -0

In [27]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}

In [28]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}